<a href="https://colab.research.google.com/github/akhilesh22210374/gen_ai/blob/main/GenAI_Lab_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install torch torchvision torchaudio
!pip install numpy librosa scipy matplotlib soundfile

In [ ]:
import torch
import torchaudio
import numpy as np
import soundfile as sf

In [ ]:
tacotron2 = torch.hub.load("nvidia/DeepLearningExamples:torchhub", "nvidia_tacotron2")
tacotron2.eval()

In [ ]:
waveglow = torch.hub.load("nvidia/DeepLearningExamples:torchhub", "nvidia_waveglow")
waveglow.eval()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tacotron2.to(device)
waveglow.to(device)

text = "Hello, this is an AI-generated voice."

sequence = tacotron2.parse_input(text).to(device)
mel_output, _, _ = tacotron2.infer(sequence)

with torch.no_grad():
    audio = waveglow.infer(mel_output)

audio_numpy = audio.squeeze().cpu().numpy()
sf.write("output.wav", audio_numpy, samplerate=22050)

print("Audio generated: output.wav")

In [ ]:
!pip install speechbrain


In [ ]:

import os
import torchaudio
from speechbrain.inference.TTS import Tacotron2
from speechbrain.inference.vocoders import HIFIGAN
from IPython.display import Audio, display

tacotron2 = Tacotron2.from_hparams(source="speechbrain/tts-tacotron2-ljspeech", savedir="tmpdir_tts")
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech", savedir="tmpdir_vocoder")

while True:
    user_text = input("Text to convert to Speech: ")
    if user_text.lower() == "exit":
        break
    mel_output, mel_length, alignment = tacotron2.encode_text(user_text)
    waveforms = hifi_gan.decode_batch(mel_output)
    output_filename = "output.wav"
    torchaudio.save(output_filename, waveforms.squeeze(1), 22050)
    print(f"Audio saved to {output_filename}\n")
    display(Audio(output_filename, rate=22050))